<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Gravitational_Slingshot_Simulation_(Python)_with_LLM_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/AhmedAbdulWahid-Data/Gravitational_Slingshot_Simulation/blob/main/Gravitational_Slingshot_Simulation.py

In [2]:
!git clone https://github.com/AhmedAbdulWahid-Data/Gravitational_Slingshot_Simulation.git

Cloning into 'Gravitational_Slingshot_Simulation'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 2.83 MiB | 34.91 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
!mv Gravitational_Slingshot_Simulation/background.jpg .
!mv Gravitational_Slingshot_Simulation/jupiter.png .

In [ ]:
import pygame
import math
import requests
import json
import time # For exponential backoff
import os   # For checking file existence

# --- Google Colab Specific API Key Setup ---
# This part assumes you are running in Google Colab and have stored your GEMINI API key as a secret.
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    print("Not in Google Colab, or 'userdata' module not found. Please set GOOGLE_API_KEY manually if needed.")
    GOOGLE_API_KEY = None # Fallback for local execution, set your key here if not using Colab secrets

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20" # Explicitly use the preview model for structured responses


pygame.init()

# --- Simulation Constants (Initial Defaults, will be overridden by LLM) ---
WIDTH, HEIGHT = 800, 600
DEFAULT_PLANET_MASS = 100
DEFAULT_SHIP_MASS = 5
G = 5 # Gravitational constant
FPS = 60
PLANET_SIZE = 50
OBJ_SIZE = 7
DEFAULT_VEL_SCALE = 100 # Scale for initial velocity from mouse input

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0) # For loading text
BLACK = (0, 0, 0) # For default background

# Load assets (ensure these images are in the same directory or provide full paths)
# For simplicity in this conceptual example, we assume they exist.
# Added FileNotFoundError to the exception handling
try:
    # Check if the file exists before attempting to load to provide a more specific message
    if not os.path.exists("background.jpg"):
        raise FileNotFoundError("background.jpg not found. Using default black background.")
    BG = pygame.transform.scale(pygame.image.load("background.jpg"), (WIDTH, HEIGHT))
except (pygame.error, FileNotFoundError) as e:
    print(f"Warning: {e}. Using a default black background.")
    BG = pygame.Surface((WIDTH, HEIGHT))
    BG.fill(BLACK) # Default to black if image not found

try:
    if not os.path.exists("jupiter.png"):
        raise FileNotFoundError("jupiter.png not found. Drawing a blue circle for the planet.")
    PLANET_IMG = pygame.transform.scale(pygame.image.load("jupiter.png"), (PLANET_SIZE * 2, PLANET_SIZE * 2))
except (pygame.error, FileNotFoundError) as e:
    print(f"Warning: {e}. Drawing a blue circle for the planet.")
    PLANET_IMG = None # Flag to draw a circle instead

# --- LLM API Call Function ---
def get_llm_parameters(prompt_text, max_retries=5, initial_delay=1.0):
    """
    Calls the Gemini LLM API to get simulation parameters.
    Implements exponential backoff for retries.
    """
    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{LLM_MODEL_NAME}:generateContent?key={GOOGLE_API_KEY}"
    headers = {"Content-Type": "application/json"}

    payload = {
        "contents": [{"role": "user", "parts": [{"text": prompt_text}]}],
        "generationConfig": {
            "responseMimeType": "application/json",
            "responseSchema": {
                "type": "OBJECT",
                "properties": {
                    "planetMass": {"type": "NUMBER", "description": "Mass of the central planet"},
                    "shipMass": {"type": "NUMBER", "description": "Mass of the spacecraft"},
                    "initialVelocityScale": {"type": "NUMBER", "description": "Scale factor for spacecraft initial velocity"},
                    "missionBrief": {"type": "STRING", "description": "A short mission brief for the current scenario"}
                },
                "required": ["planetMass", "shipMass", "initialVelocityScale", "missionBrief"]
            }
        }
    }

    retries = 0
    delay = initial_delay

    while retries < max_retries:
        try:
            response = requests.post(api_url, headers=headers, data=json.dumps(payload))
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            result = response.json()

            if result and result.get("candidates") and result["candidates"][0].get("content") and result["candidates"][0]["content"].get("parts"):
                llm_data_str = result["candidates"][0]["content"]["parts"][0]["text"]
                # LLM might sometimes return non-string JSON due to errors; ensure it's a string
                if isinstance(llm_data_str, dict): # Handle cases where it returns dict directly
                    return llm_data_str
                return json.loads(llm_data_str)
            else:
                print(f"LLM response missing expected content. Response: {result}")
                return None

        except requests.exceptions.RequestException as e:
            print(f"API request failed (Attempt {retries + 1}/{max_retries}): {e}")
            if response is not None:
                print(f"Response status: {response.status_code}, Body: {response.text}")
            retries += 1
            if retries < max_retries:
                time.sleep(delay)
                delay *= 2 # Exponential backoff
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response from LLM: {e}. Raw response: {response.text if response else 'N/A'}")
            return None
        except Exception as e:
            print(f"An unexpected error occurred during API call: {e}")
            return None

    print(f"Failed to get LLM parameters after {max_retries} retries.")
    return None

# Classes
class Planet:
    def __init__(self, x, y, mass):
        self.x = x
        self.y = y
        self.mass = mass

    def draw(self):
        if PLANET_IMG:
            # Draw the image centered
            win.blit(PLANET_IMG, (self.x - PLANET_SIZE, self.y - PLANET_SIZE))
        else:
            # Draw a circle if image not loaded
            pygame.draw.circle(win, BLUE, (int(self.x), int(self.y)), PLANET_SIZE)

class Spacecraft:
    def __init__(self, x, y, vel_x, vel_y, mass):
        self.x = x
        self.y = y
        self.vel_x = vel_x
        self.vel_y = vel_y
        self.mass = mass
        self.path = [] # Store path points for a trail

    def move(self, planet):
        distance = math.sqrt((self.x - planet.x)**2 + (self.y - planet.y)**2)
        # Avoid division by zero or extremely large forces if spacecraft is too close to planet
        if distance < 1: # Small epsilon to prevent extreme forces at 0 distance
            distance = 1

        force = (G * self.mass * planet.mass) / distance ** 2

        # Calculate acceleration (a = F/m)
        acceleration = force / self.mass

        # Angle from spacecraft to planet
        angle = math.atan2(planet.y - self.y, planet.x - self.x)

        acceleration_x = acceleration * math.cos(angle)
        acceleration_y = acceleration * math.sin(angle)

        # Update velocity and position
        self.vel_x += acceleration_x
        self.vel_y += acceleration_y
        self.x += self.vel_x
        self.y += self.vel_y

        # Add current position to path for trail
        self.path.append((int(self.x), int(self.y)))
        # Keep path length reasonable (e.g., last 100 points)
        if len(self.path) > 100:
            self.path.pop(0)

    def draw(self):
        # Draw the spacecraft
        pygame.draw.circle(win, RED, (int(self.x), int(self.y)), OBJ_SIZE)

        # Draw the trail
        if len(self.path) > 1:
            for i in range(len(self.path) - 1):
                # Draw line segment, potentially fading over time or just a solid line
                pygame.draw.line(win, (255, 0, 0), self.path[i], self.path[i+1], 1)


def create_ship(location, mouse, ship_mass_val, vel_scale_val):
    t_x, t_y = location
    m_x, m_y = mouse
    vel_x = (m_x - t_x) / vel_scale_val
    vel_y = (m_y - t_y) / vel_scale_val
    return Spacecraft(t_x, t_y, vel_x, vel_y, ship_mass_val)

def main():
    running = True
    clock = pygame.time.Clock()

    # --- LLM Integration Point: Get parameters from LLM at startup ---
    current_planet_mass = DEFAULT_PLANET_MASS
    current_ship_mass = DEFAULT_SHIP_MASS
    current_vel_scale = DEFAULT_VEL_SCALE
    current_mission_brief = "Loading scenario..."

    # Initial display to show loading
    win.blit(BG, (0, 0))
    loading_font = pygame.font.Font(None, 36)
    loading_text = loading_font.render(current_mission_brief, True, YELLOW)
    text_rect = loading_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
    win.blit(loading_text, text_rect)
    pygame.display.update()


    # Define the prompt for the LLM
    prompt_for_llm = (
        "Generate a JSON object with simulation parameters for a gravitational slingshot game. "
        "Include 'planetMass' (number between 100 and 500, no decimal), "
        "'shipMass' (number between 1 and 10, no decimal), "
        "'initialVelocityScale' (number between 50 and 200, no decimal), "
        "and a 'missionBrief' (a short, engaging string less than 100 characters, describing the spacecraft's goal). "
        "Ensure all fields are present and follow the specified types and ranges."
    )

    llm_generated_params = get_llm_parameters(prompt_for_llm)

    if llm_generated_params:
        # Safely get values, falling back to defaults if LLM somehow misses a field
        current_planet_mass = llm_generated_params.get('planetMass', DEFAULT_PLANET_MASS)
        current_ship_mass = llm_generated_params.get('shipMass', DEFAULT_SHIP_MASS)
        current_vel_scale = llm_generated_params.get('initialVelocityScale', DEFAULT_VEL_SCALE)
        current_mission_brief = llm_generated_params.get('missionBrief', "Mission: Explore the unknown!")
        print("LLM Generated Parameters:")
        print(f"  Planet Mass: {current_planet_mass}")
        print(f"  Ship Mass: {current_ship_mass}")
        print(f"  Velocity Scale: {current_vel_scale}")
        print(f"  Mission: {current_mission_brief}")
    else:
        print("Failed to get LLM parameters, using default values.")
        current_mission_brief = "Mission: Default Exploration!"


    # Initialize objects with (potentially) LLM-generated parameters
    planet = Planet(WIDTH // 2, HEIGHT // 2, current_planet_mass)
    objects = []
    temp_obj_pos = None

    # Set window title to include a hint from the mission brief
    pygame.display.set_caption(f"Slingshot: {current_mission_brief[:70]}...") # Truncate for title

    # Font for displaying mission brief in-game
    font = pygame.font.Font(None, 24) # Default font, size 24

    while running:
        clock.tick(FPS)

        # Handle events
        mouse_pos = pygame.mouse.get_pos()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if event.type == pygame.MOUSEBUTTONDOWN:
                if temp_obj_pos:
                    # Use LLM-generated ship mass and velocity scale
                    obj = create_ship(temp_obj_pos, mouse_pos, current_ship_mass, current_vel_scale)
                    objects.append(obj)
                    temp_obj_pos = None
                else:
                    temp_obj_pos = mouse_pos

        # Draw background
        win.blit(BG, (0, 0))

        # Preview new ship trajectory
        if temp_obj_pos:
            pygame.draw.line(win, WHITE, temp_obj_pos, mouse_pos, 2)
            pygame.draw.circle(win, RED, temp_obj_pos, OBJ_SIZE)

        # Move and draw objects
        for obj in objects[:]: # Iterate over a slice to allow removal during iteration
            obj.move(planet)
            obj.draw()

            # Remove objects if off-screen or colliding with the planet
            off_screen = obj.x < -100 or obj.x > WIDTH + 100 or obj.y < -100 or obj.y > HEIGHT + 100 # Give some buffer
            collided = math.sqrt((obj.x - planet.x)**2 + (obj.y - planet.y)**2) <= PLANET_SIZE

            if off_screen or collided:
                objects.remove(obj)

        # Draw the planet
        planet.draw()

        # Render and display the mission brief
        text_surface = font.render(f"Mission: {current_mission_brief}", True, WHITE)
        win.blit(text_surface, (10, 10)) # Top-left corner

        # Display instructions
        instructions_font = pygame.font.Font(None, 20)
        instructions_text = instructions_font.render("Click once for launch point, click again for velocity vector. Press ESC to quit.", True, WHITE)
        win.blit(instructions_text, (10, HEIGHT - 30))

        # Update the screen
        pygame.display.update()

    pygame.quit()

if __name__ == "__main__":
    win = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Gravitational Slingshot Effect") # Initial title, will be updated by LLM
    main()

Google Generative AI configured successfully using Colab Secrets.
LLM Generated Parameters:
  Planet Mass: 350
  Ship Mass: 5
  Velocity Scale: 120
  Mission: Navigate the asteroid field to reach the distant colony!
